In [1]:
import numpy as np
import skopt
from tensorflow import keras
from tqdm.auto import tqdm

import src.LoadData as LoadData
import src.ModelBuilder as ModelBuilder
import src.TrainProdecure as TrainProdecure

In [2]:
class tqdm_skopt:
    def __init__(self, **kwargs):
        self._bar = tqdm(**kwargs)

    def __call__(self, res):
        self._bar.update()


In [3]:
# General Configurations:
EPOCHS_PER_TRAINING = 15
# DATASETS = LoadData.get_all_datasets_test_train_np_arrays('../datasets/')
DATASETS = LoadData.get_mcdcnn_datasets_test_train_np_arrays('../datasets/')
ds_names = list(DATASETS.keys())

ds_names

['electric_devices', 'freezers', 'sony_robot']

In [5]:
def get_config_str(nr_conv_layers=2, nr_conv_filters=8, kernel_size=5, padding_method="same",
                 dense_size=732, nr_dense_layers=1,
                 conv_activation="sigmoid", dense_activation="sigmoid",
                 pooling_method=keras.layers.MaxPool1D, pooling_size=2,
                 dropout_rate=0.5, use_batchnorm=False):
    return f"{nr_conv_layers}-conv-layers_{nr_conv_filters}-filters_{kernel_size}-kernel-size_{padding_method}-padding_({conv_activation})--{nr_dense_layers}-dense_layers_({dense_activation})--{pooling_method}_({pooling_size})_{dropout_rate if dropout_rate>0 else ''}{'_batchnorm' if use_batchnorm else ''}"

In [6]:
def train_and_evaluate_config(config):
    config_string = get_config_str(*config)
    print(f"Training model with config: {config_string}")

    accuracies_per_dataset = []

    try:
        for ds_name in (t := tqdm(ds_names, unit="dataset", desc="Training the model on each dataset...")):
            t.set_postfix_str(f"Dataset: {ds_name}")
            train_x, train_y = DATASETS[ds_name]["train_data"]

            input_size = train_x.shape[1]
            output_size = len(np.unique(train_y))
            model = ModelBuilder.build_MCDCNN(input_size, output_size, *config)

            _, history = TrainProdecure.train_single_model(model, train_x, train_y, epochs=EPOCHS_PER_TRAINING, model_name=f"{ds_name}_{config_string}", dataset_name=ds_name)

            val_acc = history.history["val_accuracy"][-1]
            accuracies_per_dataset.append(val_acc)
    except Exception as e:
        print(f"Error while training model with config {config_string}: {e}")
        return 1

    return 1 - np.average(accuracies_per_dataset)

In [7]:
result = skopt.gp_minimize(
    train_and_evaluate_config,
    [
        skopt.space.Integer(1, 4, name="nr_conv_layers"),
        skopt.space.Integer(2, 10, name="nr_conv_filters"),
        skopt.space.Integer(3, 7, name="kernel_size"),
        skopt.space.Categorical(["same", "causal"], name="padding_method"),
        skopt.space.Integer(10, 1000, prior="log-uniform", name="dense_size"),
        skopt.space.Integer(1, 4, name="nr_dense_layers"),
        skopt.space.Categorical(["sigmoid", "relu"], name="conv_activation"),
        skopt.space.Categorical(["sigmoid", "relu"], name="dense_activation"),
        skopt.space.Categorical(["max_pool", "avg_pool"], name="pooling_method"),
        skopt.space.Integer(2, 4, name="pooling_size"),
        skopt.space.Categorical(list(np.arange(0, 1, 0.1)), name="dropout_rate"),
        skopt.space.Categorical([True, False], name="use_batchnorm"),
    ],
    n_calls=100,
    n_random_starts=5,
    verbose=True,
    random_state=42,
    n_jobs=1,
    callback=[tqdm_skopt(total=500, desc="Optimizing Hyperparameters")]
    # callback=[skopt.callbacks.CheckpointSaver("./checkpoints/checkpoint.pkl")],
)

Optimizing Hyperparameters:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration No: 1 started. Evaluating function at random point.
Training model with config: 3-conv-layers_3-filters_6-kernel-size_same-padding_(relu)--1-dense_layers_(relu)--avg_pool_(3)_0.1_batchnorm


Training the model on each dataset...:   0%|          | 0/3 [00:00<?, ?dataset/s]

Training electric_devices_3-conv-layers_3-filters_6-kernel-size_same-padding_(relu)--1-dense_layers_(relu)--av…

Training freezers_3-conv-layers_3-filters_6-kernel-size_same-padding_(relu)--1-dense_layers_(relu)--avg_pool_(…

...

Training the model on each dataset...:   0%|          | 0/3 [00:00<?, ?dataset/s]

Training electric_devices_2-conv-layers_10-filters_7-kernel-size_same-padding_(relu)--2-dense_layers_(relu)--m…

Training freezers_2-conv-layers_10-filters_7-kernel-size_same-padding_(relu)--2-dense_layers_(relu)--max_pool_…

Training sony_robot_2-conv-layers_10-filters_7-kernel-size_same-padding_(relu)--2-dense_layers_(relu)--max_poo…

Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 70.4898
Function value obtained: 0.1148
Current minimum: 0.0963


In [8]:
result

          fun: 0.09630459547042847
            x: [2, 9, 7, 'same', 196, 2, 'relu', 'relu', 'max_pool', 2, 0.1, True]
    func_vals: [ 1.506e-01  6.633e-01 ...  1.590e-01  1.148e-01]
      x_iters: [[3, 3, 6, 'same', 78, 1, 'relu', 'relu', 'avg_pool', 3, 0.1, True], [4, 2, 7, 'same', 167, 1, 'relu', 'sigmoid', 'avg_pool', 2, 0.9, False], [1, 7, 5, 'same', 86, 4, 'sigmoid', 'relu', 'avg_pool', 4, 0.5, False], [1, 4, 4, 'same', 166, 3, 'relu', 'relu', 'avg_pool', 4, 0.4, False], [3, 2, 6, 'causal', 62, 4, 'sigmoid', 'relu', 'max_pool', 3, 0.9, True], [1, 10, 3, 'causal', 1000, 4, 'relu', 'relu', 'max_pool', 4, 0.9, False], [1, 4, 3, 'same', 255, 4, 'sigmoid', 'relu', 'max_pool', 3, 0.4, True], [1, 10, 3, 'same', 1000, 4, 'relu', 'relu', 'max_pool', 4, 0.9, False], [4, 6, 7, 'causal', 109, 2, 'relu', 'relu', 'avg_pool', 2, 0.2, True], [1, 10, 3, 'causal', 10, 1, 'relu', 'relu', 'avg_pool', 4, 0.30000000000000004, False], [2, 5, 3, 'same', 79, 4, 'relu', 'sigmoid', 'max_pool', 4, 0.7000000

In [28]:
# name="nr_conv_layers"),
#  name="nr_conv_filters"),
# name="kernel_size"),
# same", "causal"], name="padding_method"),
# 00, prior="log-uniform", name="dense_size"),
# name="nr_dense_layers"),
# sigmoid", "relu"], name="conv_activation"),
# sigmoid", "relu"], name="dense_activation"),
# max_pool", "avg_pool"], name="pooling_method"),
# name="pooling_size"),
# st(np.arange(0, 1, 0.1)), name="dropout_rate"),
# rue, False], name="use_batchnorm"),
def convert_np(x):
    if isinstance(x, np.integer):
        return int(x)
    elif isinstance(x, np.floating):
        return float(x)
    elif isinstance(x, np.bool_):
        return bool(x)
    else:
        return x

key_list = ["nr_conv_layers", "nr_conv_filters", "kernel_size", "padding_method", "dense_size", "nr_dense_layers", "conv_activation", "dense_activation", "pooling_method", "pooling_size", "dropout_rate", "use_batchnorm"]
values = [convert_np(a) for a in result.x]
result_dict = dict(zip(key_list, values))

print(f"Best configuration: {result_dict} \nresults in {(1-result.fun)*100:.5f}% accuracy")

Best configuration: {'nr_conv_layers': 2, 'nr_conv_filters': 9, 'kernel_size': 7, 'padding_method': 'same', 'dense_size': 196, 'nr_dense_layers': 2, 'conv_activation': 'relu', 'dense_activation': 'relu', 'pooling_method': 'max_pool', 'pooling_size': 2, 'dropout_rate': 0.1, 'use_batchnorm': True} 
results in 90.36954% accuracy


[2, 9, 7, 'same', 196, 2, 'relu', 'relu', 'max_pool', 2, 0.1, True]

In [29]:
import json
json.dump(result_dict, open("../results/best_config_for_mcdcnn.json", "w"))